In [12]:
import os
import warnings
import findfile
import pandas as pd
import json
import transformers
import torch

In [2]:
print(transformers.__version__)

c:\Users\hongk\Documents\GitHub\ta-final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv-absa-3.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.28.0


In [4]:
from pyabsa import ABSAInstruction as absa_instruction

Version 0.9.10.post1 of metric_visualizer is outdated. Version 0.9.13.post1 was released Wednesday May 01, 2024.
[2024-06-07 19:50:33] (2.4.1.post1) PyABSA(2.4.1.post1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






c:\Users\hongk\Documents\GitHub\ta-final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv-absa-3.9\lib\site-packages\tensorflow\python\framework\dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
c:\Users\hongk\Documents\GitHub\ta-final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv-absa-3.9\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
C:\Users\hongk\AppData\Local\Programs\Python\Python39\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


In [5]:
warnings.filterwarnings("ignore")

In [6]:
task_name = "multitask"
experiment_name = "instruction"
# model_checkpoint = "google/flan-t5-base"
model_checkpoint = "kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined"

In [7]:
print("Experiment Name: ", experiment_name)
model_out_path = "checkpoints"
model_out_path = os.path.join(
    model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}"
)
print("Model output path: ", model_out_path)

Experiment Name:  instruction
Model output path:  checkpoints\multitask\kevinscariaate_tk-instruct-base-def-pos-neg-neut-combined-instruction


In [17]:
id_train_file_path = "./ABSADatasets/datasets/acos_datasets/504.Restaurant16/"
id_test_file_path = "./ABSADatasets/datasets/acos_datasets/504.Restaurant16/"

In [18]:
import json

In [19]:
id_tr_df = absa_instruction.data_utils.read_json(id_train_file_path, "train")
id_te_df = absa_instruction.data_utils.read_json(id_test_file_path, "test")

./ABSADatasets/datasets/acos_datasets/504.Restaurant16/rest16_quad_train.tsv.jsonl
./ABSADatasets/datasets/acos_datasets/504.Restaurant16/rest16_quad_test.tsv.jsonl


In [20]:
id_tr_df = pd.DataFrame(id_tr_df)
id_tr_df

,text,labels
0,judging from previous posts this used to be a ...,"[{'aspect': 'place', 'opinion': 'not any longe..."
1,"we , there were four of us , arrived at noon -...","[{'aspect': 'staff', 'opinion': 'rude', 'polar..."
2,"they never brought us complimentary noodles , ...","[{'aspect': 'NULL', 'opinion': 'NULL', 'polari..."
3,the food was lousy - too sweet or too salty an...,"[{'aspect': 'food', 'opinion': 'lousy', 'polar..."
4,"after all that , they complained to me about t...","[{'aspect': 'NULL', 'opinion': 'complained', '..."
...,...,...
1525,"i ca n ' t believe that it was , but please pu...","[{'aspect': 'NULL', 'opinion': 'NULL', 'polari..."
1526,the waitress came to check in on us every few ...,"[{'aspect': 'waitress', 'opinion': 'NULL', 'po..."
1527,i could n ' t ignore the fact that she reach o...,"[{'aspect': 'NULL', 'opinion': 'NULL', 'polari..."
1528,she then put the check down without asking if ...,"[{'aspect': 'NULL', 'opinion': 'NULL', 'polari..."


In [21]:
id_te_df = pd.DataFrame(id_te_df)
id_te_df

,text,labels
0,yum !,"[{'aspect': 'NULL', 'opinion': 'yum', 'polarit..."
1,serves really good sushi .,"[{'aspect': 'sushi', 'opinion': 'good', 'polar..."
2,not the biggest portions but adequate .,"[{'aspect': 'portions', 'opinion': 'not the bi..."
3,green tea creme brulee is a must !,"[{'aspect': 'green tea creme brulee', 'opinion..."
4,it has great sushi and even better service .,"[{'aspect': 'sushi', 'opinion': 'great', 'pola..."
...,...,...
578,insultingly overpriced,"[{'aspect': 'NULL', 'opinion': 'overpriced', '..."
579,- mediocre service / quality,"[{'aspect': 'service', 'opinion': 'mediocre', ..."
580,to their credit they removed the dish from the...,"[{'aspect': 'manager', 'opinion': 'NULL', 'pol..."
581,two rascally kids were seated near us for the ...,"[{'aspect': 'NULL', 'opinion': 'NULL', 'polari..."


In [22]:
loader = absa_instruction.data_utils.InstructDatasetLoader(id_tr_df, id_te_df)

In [23]:
if loader.train_df_id is not None:
    loader.train_df_id = loader.prepare_instruction_dataloader(loader.train_df_id)
if loader.test_df_id is not None:
    loader.test_df_id = loader.prepare_instruction_dataloader(loader.test_df_id)
# if loader.train_df_ood is not None:
#     loader.train_df_ood = loader.prepare_instruction_dataloader(loader.train_df_ood)
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.prepare_instruction_dataloader(loader.test_df_ood)

In [24]:
# Create T5 utils object
t5_exp = absa_instruction.model.T5Generator(model_checkpoint)

[2024-06-07 19:59:25] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-06-07 19:59:25] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-06-07 19:59:25] (2.4.1.post1) Checkpoint:kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined is not found, you can raise an issue for requesting shares of checkpoints
[2024-06-07 19:59:25] (2.4.1.post1) No checkpoint found in Model Hub for task: kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined


In [25]:
# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map: 100%|██████████| 2135/2135 [00:00<00:00, 10544.48 examples/s]


In [26]:
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map: 100%|██████████| 2135/2135 [00:00<00:00, 11008.33 examples/s]


In [27]:
training_args = {
    "output_dir": model_out_path,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 8,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "load_best_model_at_end": True,
    "push_to_hub": False,
    "eval_accumulation_steps": 1,
    "predict_with_generate": True,
    "logging_steps": 1000000000,
    "use_mps_device": False,
    # 'fp16': True,
    "fp16": False,
}


In [28]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Trainer device: cuda:0

Model training started ....


  2%|▏         | 205/11208 [1:08:53<301:43:07, 98.72s/it] 

KeyboardInterrupt: 

In [ ]:
# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="train",
    batch_size=16,
)
id_tr_labels = [i.strip() for i in id_ds["train"]["labels"]]

In [ ]:
id_te_pred_labels = t5_exp.get_labels(
    predictor=model_trainer,
    tokenized_dataset=id_tokenized_ds,
    sample_set="test",
    batch_size=16,
)
id_te_labels = [i.strip() for i in id_ds["test"]["labels"]]

In [ ]:
metrics = t5_exp.get_classic_metrics(id_tr_labels, id_tr_pred_labels)
print("----------------------- Classic Training Set Metrics -----------------------")
print(metrics)

In [ ]:
metrics = t5_exp.get_classic_metrics(id_te_labels, id_te_pred_labels)
print("----------------------- Classic Testing Set Metrics -----------------------")
print(metrics)